## Background

The last week and a half we have been having issues with the Hall probes stopping reading soon after starting a run. This never happened until switching to doing slow controls and Hall probe (CERN software) at the same time. After swapping out many of the components with spares, we tried plugging Hall probes directly in to the DAQ computer. It ran without stopping overnight, so Thomas will punch a hole in the wall (or add to existing hole) and run the TrippLite USB repeater (check name). This will give two USB lines into the enclosure: one for the Hall probes, one for everything else (on USB hub).

This data won't be all that interesting (no magnetic field), but I think I may be able to use it to confirm or refute Thomas' claim that there is a temperature lag (see docdb-**###**)

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import datetime
from dateutil import parser
from datetime import datetime
from sys import getsizeof
import re

import operator
from scipy import stats

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.rcParams['axes.axisbelow'] = True

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
init_notebook_mode(True)

import seaborn as sns

from hallprobecalib import hpc_ext_path
from hallprobecalib.hpcplots import scatter2d,scatter3d,histo,py_profile

In [ ]:
plotdir = hpc_ext_path+'plots/hall_probe_stability/2019-08-27/'

In [ ]:
pkl_dir = hpc_ext_path+"datafiles/hall_probe_stability/"
#pkl_file_slow_old = "hall_no_magnet.run01.raw.base.pkl"
pkl_file_slow = "hall_no_magnet.run01.raw.slow.pkl"
pkl_file_fast = "hall_no_magnet.run01.raw.fast.pkl"
pkl_file_fast_sparse = "hall_no_magnet.run02.sparse.fast.pkl"

In [ ]:
df_slow = pd.read_pickle(pkl_dir+pkl_file_slow)

In [ ]:
df_fast = pd.read_pickle(pkl_dir+pkl_file_fast)
#df_fast_sparse = pd.read_pickle(pkl_dir+pkl_file_fast_sparse)

In [ ]:
#df_fast.tail(1)

In [ ]:
#df_slow.tail(1)

## Cleaning

In [ ]:
directory = "/home/ckampa/Dropbox/LogFiles/"#hpc_ext_path+'datafiles/magnet_stability/nmr/'
filename_slow = '2019-08-27 132923slow.txt'
filename_fast = '2019-08-27 132800.txt'

In [ ]:
fast_cols_init = ["Date", "Time", 
                  "1", "1_Bx", "1_By", "1_Bz", "1_T",
                  "2", "2_Bx", "2_By", "2_Bz", "2_T",
                  "3", "3_Bx", "3_By", "3_Bz", "3_T",]
                  #"4", "4_Bx", "4_By", "4_Bz", "4_T",
                  #"5", "5_Bx", "5_By", "5_Bz", "5_T",]

In [ ]:
df_slow = pd.read_csv(directory+filename_slow)

In [ ]:
df_slow.head()

In [ ]:
df_fast = pd.read_csv(directory+filename_fast, skiprows=7, delim_whitespace=True, names=fast_cols_init)

In [ ]:
df_fast.head()

### Cleaning Fast Data Set

In [ ]:
df_fast.tail(1)

In [ ]:
df_fast.tail(1)

In [ ]:
#df_fast

In [ ]:
'''fast_cols_init = ["Date", "Time", 
                  "1", "1_Bx", "1_By", "1_Bz", "1_T",
                  "2", "2_Bx", "2_By", "2_Bz", "2_T",
                  "3", "3_Bx", "3_By", "3_Bz", "3_T",
                  "4", "4_Bx", "4_By", "4_Bz", "4_T",
                  "5", "5_Bx", "5_By", "5_Bz", "5_T",]''';

In [ ]:
#len(fast_cols_init)

In [ ]:
#len(df_fast.columns)

In [ ]:
# raw values on every other line....
df_fast_cal = df_fast[::2]
df_fast_raw = df_fast[1::2]

In [ ]:
df_fast_cal.reset_index(drop=True, inplace=True)
df_fast_raw.reset_index(drop=True, inplace=True)

In [ ]:
#df_fast_cal

In [ ]:
#df_fast_raw

In [ ]:
#dates = [parser.parse(row.Date+" "+row.Time[:-4]) for row in df_fast_cal.itertuples()]
dates = [datetime.strptime(row.Date+" "+row.Time, "%Y-%m-%d %H:%M:%S:%f") for row in df_fast_cal.itertuples()]

In [ ]:
df_fast_cal["Datetime"] = pd.to_datetime(dates)
df_fast_cal = df_fast_cal.set_index("Datetime")

In [ ]:
#dates = [parser.parse(row.Date+" "+row.Time[:-4]) for row in df_fast_cal.itertuples()]
dates = [datetime.strptime(row.Date[1:]+" "+row.Time, "%Y-%m-%d %H:%M:%S:%f") for row in df_fast_raw.itertuples()]

In [ ]:
df_fast_raw["Datetime"] = pd.to_datetime(dates)
df_fast_raw = df_fast_raw.set_index("Datetime")

In [ ]:
df_fast_raw.drop(columns=["Date","Time"], inplace=True)

In [ ]:
df_fast_cal.drop(columns=["Date","Time"], inplace=True)

In [ ]:
raw_col_dict = {i: "Raw_"+i for i in df_fast_raw.columns}
cal_col_dict = {i: "Cal_"+i for i in df_fast_cal.columns}

In [ ]:
df_fast_cal.rename(columns=cal_col_dict, inplace=True)
df_fast_raw.rename(columns=raw_col_dict, inplace=True)

In [ ]:
df_fast = pd.concat([df_fast_cal, df_fast_raw], axis=1)

### Slow Control Data Cleaning

In [ ]:
dates = [parser.parse(row.Time) for row in df_slow.itertuples()]# if isdatetime(row.Time.values[0])]

In [ ]:
df_slow["Datetime"] = pd.to_datetime(dates)#.astype("str")
df_slow = df_slow.set_index("Datetime")

There is no Hall probe data.

In [ ]:
df_fast = df_fast.loc[:df_slow.index[-1]]

In [ ]:
df_fast.loc[:,"NMR"] = [df_slow.iloc[df_slow.index.get_loc(row, method='nearest')][" NMR [T]"] for row in df_fast.index]

In [ ]:
for i in range(1,3):
    df_fast.eval(f"Cal_{i}_Bmag = (Cal_{i}_Bx**2+Cal_{i}_By**2+Cal_{i}_Bz**2)**(1/2)", inplace=True)
    df_fast.eval(f"Cal_{i}_Res = (NMR - Cal_{i}_Bmag)", inplace=True)
    df_fast.eval(f"Cal_{i}_Res_Rel = (NMR - Cal_{i}_Bmag) / NMR", inplace=True)

for i in range(1,3):
    for B in ["Bx","By","Bz"]:
        #df_fast.eval(f"Cal_{i}_Bmag = (Cal_{i}_Bx**2+Cal_{i}_By**2+Cal_{i}_Bz**2)**(1/2)", inplace=True)
        df_fast.eval(f"Cal_{i}_{B}_Res = (NMR - Cal_{i}_{B})", inplace=True)
        df_fast.eval(f"Cal_{i}_{B}_Res_Rel = (NMR - Cal_{i}_{B}) / NMR", inplace=True)


t0 = pd.to_timedelta(df_fast.index.values)[0] / pd.offsets.Second(1)

df_fast["seconds_delta"] = pd.to_timedelta(df_fast.index.values) / pd.offsets.Second(1) - t0#1565697533.416
df_slow["seconds_delta"] = pd.to_timedelta(df_slow.index.values) / pd.offsets.Second(1) - t0#1565697533.416

In [ ]:
for i in range(1,3):
    df_fast[f"Cal_{i}_Transverse"] = (df_fast[f"Cal_{i}_Bx"]**2+df_fast[f"Cal_{i}_By"]**2)**(1/2)
    df_fast[f"Cal_{i}_Theta"] = np.arccos(df_fast[f"Cal_{i}_Bz"]/df_fast[f"Cal_{i}_Bmag"])
    df_fast[f"Cal_{i}_Phi"] = np.arccos(df_fast[f"Cal_{i}_Bx"]/df_fast[f"Cal_{i}_Transverse"])

In [ ]:
df_slow.to_pickle(pkl_dir+pkl_file_slow)

In [ ]:
df_fast.to_pickle(pkl_dir+pkl_file_fast)

In [ ]:
col_drop = [i for i in df_fast.columns if ("1" not in i) and ("NMR" not in i)]

In [ ]:
#col_drop

In [ ]:
df_fast_sparse = df_fast.drop(columns=col_drop)

In [ ]:
df_fast_sparse.to_pickle(pkl_dir+pkl_file_fast_sparse)

In [ ]:
getsizeof(df_slow) / 1000/1000, getsizeof(df_fast) / 1000/1000, getsizeof(df_fast_sparse) / 1000/1000

## Plotting

### Temperature Lag

In [ ]:
fig = scatter2d(3*[df_fast[::10].index], [df_fast[col][::10] for col in ["Raw_1_Bx","Raw_1_By","Raw_1_Bz",]], inline=True)

In [ ]:
fig = scatter2d(df_fast[::10].index, df_fast[::10]["Cal_1_T"])

In [ ]:
fig = scatter2d(3*[df_fast[::10].index], [df_fast[col][::10] for col in ["Cal_1_Bx","Cal_1_By","Cal_1_Bz",]], inline=True)

In [ ]:
#df_t = df_fast.loc["2019-08-27 20:15:00":"2019-08-27 23:00:00"]
#df_t = df_fast.loc["2019-08-27 18:00:00":"2019-08-28 00:00:00"]
df_t = df_fast.loc["2019-08-27 19:00:00":"2019-08-27 21:00:00"]

In [ ]:
len(df_t)

In [ ]:
fig = scatter2d(df_t.index, df_t["Cal_1_T"])

In [ ]:
fig = scatter2d(3*[df_t.index], [df_t[col] for col in ["Raw_1_Bx","Raw_1_By","Raw_1_Bz",]], inline=True)

In [ ]:
for B in ["1","2","3"]:
    fig = scatter2d(df_t.index, df_t[f"Cal_{B}_T"])
    fig = scatter2d(3*[df_t.index], [df_t[col] for col in [f"Raw_{B}_Bx",f"Raw_{B}_By",f"Raw_{B}_Bz",]], inline=True)

In [ ]:
for B in ["1","2","3"]:
    fig = scatter2d(3*[df_t.index], [df_t[col] for col in [f"Cal_{B}_Bx",f"Cal_{B}_By",f"Cal_{B}_Bz",]], inline=True)

In [ ]:
fig = scatter2d(3*[df_t.index], [df_t[col] for col in [f"Cal_1_T",f"Cal_2_T",f"Cal_3_T",]], inline=True)

In [ ]:
fig = scatter2d(2*[df_slow.index]+3*[df_fast.index[::10]],[df_slow["Desk Ambient"],df_slow[" Spare Ambient on desk"]]+[df_fast[::10][col] for col in [f"Cal_1_T",f"Cal_2_T",f"Cal_3_T",]])

In [ ]:
for B in ["Cal_1_T","Cal_2_T","Cal_3_T"]:
    df_slow.loc[:,B] = [df_fast.iloc[df_fast.index.get_loc(row, method='nearest')][B] for row in df_slow.index]

In [ ]:
fig = scatter2d(5*[df_slow.index],[df_slow["Desk Ambient"],df_slow[" Spare Ambient on desk"]]+[df_slow[col] for col in [f"Cal_1_T",f"Cal_2_T",f"Cal_3_T",]])

In [ ]:
fig = scatter2d(df_slow.index, res)

In [ ]:
# residuals

In [ ]:
res = df_slow[f"Cal_1_T"]-df_slow["Desk Ambient"]
res.name = f"Res_1"
fig = histo(res, bins=50)

In [ ]:
# correlations

In [ ]:
fig = scatter2d([df_slow["Desk Ambient"][25:]],[df_slow[f"Cal_1_T"][25:]],lines=False,markers=True)

In [ ]:
for i in [1,2,3]:
    fig = scatter2d([df_slow[25:]["Desk Ambient"]],[df_slow[25:][f"Cal_{i}_T"]],lines=False,markers=True)

In [ ]:
def color_scatter2d(x,y,ind,time=df_slow.index,show_plot=True):
    data = [
        go.Scatter(
            x=x,
            y=y,
            marker=dict(
                size=2,
                cmax=ind.max(),
                cmin=ind.min(),
                color=ind,
                colorbar=dict(
                    title='time (index)',
                    #ticktext=df.index
                ),
                colorscale='Viridis',
            ),
            mode='markers',
            text=time,
        )
    ]
    
    layout = go.Layout(
        title = f"{y.name} vs. {x.name}",
        xaxis = dict(
            title = f"{x.name}"
        ),
        yaxis = dict(
            title = f"{y.name}"
        ),
    )

    fig = go.Figure(data=data,layout=layout)
    if show_plot:
        iplot(fig)
    return fig

In [ ]:
for i in [1,2,3]:
    fig = color_scatter2d(df_slow[25:]["Desk Ambient"],df_slow[25:][f"Cal_{i}_T"],ind=df_slow[25:].seconds_delta)

In [ ]:
# probe 2
i = 2
start = 25*60
for B in ["Bx","By","Bz"]:
    fig = color_scatter2d(df_fast[start::10][f"Cal_{i}_T"],df_fast[start::10][f"Raw_{i}_{B}"],ind=df_fast[start::10].seconds_delta, time=df_fast[start::10].index)

In [ ]:
fig = scatter2d([df_t.seconds_delta], [df_t[col] for col in ["Raw_1_By",]], show_plot=False)

In [ ]:
figprof = py_profile(df_t.seconds_delta, df_t.Raw_1_By, x_bins=50, show_plot=False)

#figprof2 = py_profile(df_slow.seconds_delta, df_slow[" Hall Element"], x_bins=50,xrange=[df_fast.seconds_delta.min(),df_fast.seconds_delta.max()], show_plot=False)

fig.add_traces([figprof.data[0],])

### First Attempt

In [ ]:
len(df_fast)

In [ ]:
df_fast.columns

In [ ]:
df_fast.eval("Cal_1_Bmag = (Cal_1_Bx**2+Cal_1_By**2+Cal_1_Bz**2)**(1/2)", inplace = True)

In [ ]:
N = 10#30#10#20#10#30#60
df_fast.loc[:, "Cal_1_Bmag_Avg"] = df_fast["Cal_1_Bmag"].rolling(window = N, center=True).mean()
df_fast.loc[:, "Cal_1_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast["Cal_1_Bmag_Avg"])/df_fast["NMR"]

In [ ]:
df_fast_5 = df_fast[N // 2::N]

In [ ]:
#df_fast_5.loc[:, "Cal_Mag_Avg"] = [df_fast.iloc[i- for i in range(len(df_fast_5))]

In [ ]:
### 60 second

In [ ]:
fig = scatter2d([df_fast.index,df_fast.index,df_fast_5.index], [df_fast.Cal_1_Bmag,df_fast.Cal_1_Bmag_Avg,df_fast_5.Cal_1_Bmag_Avg], inline=True)

In [ ]:
fig = histo(df_fast.Cal_1_Bmag, bins=50, inline=True)
fig = histo(df_fast.Cal_1_Bmag_Avg, bins=50, inline=True)
fig = histo(df_fast_5.Cal_1_Bmag_Avg, bins=50, inline=True)

In [ ]:
#df_fast.loc[:,"NMR"] = [df_slow.iloc[df_slow.index.get_loc(row, method='nearest')][" NMR [T]"] for row in df_fast.index]

In [ ]:
df_slow[" NMR [T]"].hist(bins=50)

In [ ]:
df_fast.eval("Cal_Res = (NMR - Cal_1_Bmag)", inplace=True)
df_fast.eval("Cal_Res_Rel = (NMR - Cal_1_Bmag) / NMR", inplace=True)

In [ ]:
fig = histo(df_fast.Cal_Res_Rel, bins=50, inline=True)
fig = histo(df_fast.Cal_1_Bmag_Avg_Res_Rel, bins=50, inline=True)
fig = histo(df_fast_5.Cal_1_Bmag_Avg_Res_Rel, bins=50, inline=True)
#fig = histo(df_fast.Cal_1_Bmag_Avg_Res_Rel, bins=50, inline=True)

In [ ]:
### 60 second average

In [ ]:
fig = histo([df_fast.Cal_1_Bmag,df_fast.Cal_1_Bmag_Avg], bins=50,same_bins=True, show_plot=False, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)
#fig = histo([df_fast.Cal_Res_Rel,df_fast_5.Cal_1_Bmag_Avg_Res_Rel], bins=50,same_bins=True, show_plot=False, inline=True)
#fig.update_traces(histnorm='probability density')
#iplot(fig)

In [ ]:
fig = histo([df_fast.Cal_Res_Rel,df_fast.Cal_1_Bmag_Avg_Res_Rel], bins=50,same_bins=True, show_plot=False, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)
fig = histo([df_fast.Cal_Res_Rel,df_fast_5.Cal_1_Bmag_Avg_Res_Rel], bins=50,same_bins=True, show_plot=False, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)

In [ ]:
fig = histo([df_fast.Cal_Res_Rel,df_fast.Cal_1_Bmag_Avg_Res_Rel], bins=50,same_bins=True, show_plot=False, inline=True)
#fig.update_traces(histnorm='probability density')
iplot(fig)
fig = histo([df_fast.Cal_Res_Rel,df_fast_5.Cal_1_Bmag_Avg_Res_Rel], bins=50,same_bins=True, show_plot=False, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)
#fig = histo(df_fast.Cal_1_Bmag_Avg_Res_Rel, bins=50, inline=True)
#fig = histo(df_fast_5.Cal_1_Bmag_Avg_Res_Rel, bins=10, inline=True)
#fig = histo(df_fast.Cal_1_Bmag_Avg_Res_Rel, bins=50, inline=True)

In [ ]:
(7.09-16.74)/((7.09+16.74)/2)

In [ ]:
(7.09-16.74)/(16.74)

In [ ]:
(5.77-7.09)/((5.77+7.09)/2)

In [ ]:
(5.77-7.09)/(7.09)

In [ ]:
fig = histo(df_slow[" NMR [T]"], bins=10, inline=True)

In [ ]:
df_slow[" NMR [T]"].plot()

In [ ]:
fig = scatter2d(df_slow.index, df_slow[" NMR [T]"], inline=True)

In [ ]:
import copy

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
Ns = [5,10,30,60, 120, 240, 480, 960, 1930,1930*2, 1930*4, 1930*8]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_fast.loc[:, "Cal_1_Bmag_Avg"] = df_fast["Cal_1_Bmag"].rolling(window = N, center=True).mean()
    df_fast.loc[:, "Cal_1_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast["Cal_1_Bmag_Avg"])/df_fast["NMR"]
    df_fast_N.append(copy.copy(df_fast[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df["Cal_1_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])

In [ ]:
df_fast_N[0]["Cal_1_Bmag_Avg_Res_Rel"].describe()

In [ ]:
fig = scatter2d([df_fast.index, df_fast_N[-2].index, df_slow.index], [df_fast.Cal_1_Bmag,df_fast_N[-2].Cal_1_Bmag_Avg, df_slow[" NMR [T]"]], inline=True)

In [ ]:
fig = histo([df_fast.Cal_Res_Rel]+[df.Cal_1_Bmag_Avg_Res_Rel for df in df_fast_N], bins=50,same_bins=True, show_plot=False, opacity=0.5, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)

In [ ]:
fig = histo([df_fast.Cal_Res_Rel]+[df.Cal_1_Bmag_Avg_Res_Rel for df in [df_fast_N[0]]], bins=50,same_bins=False, show_plot=False, opacity=0.5, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)

In [ ]:
fig = histo([df.Cal_1_Bmag_Avg_Res_Rel for df in [df_fast_N[0]]], bins=50,same_bins=False, show_plot=True, opacity=0.5, inline=True)

In [ ]:
df_fast_N[0]["Cal_1_Bmag_Avg_Res_Rel"].describe()

In [ ]:
getsizeof(df_fast_N[0]) / 1000 / 1000

### Noise on all Hall probes

In [ ]:
#df_fast.columns

In [ ]:
'''for i in range(1,6):
    df_fast.eval(f"Cal_{i}_Bmag = (Cal_{i}_Bx**2+Cal_{i}_By**2+Cal_{i}_Bz**2)**(1/2)", inplace=True)
    df_fast.eval(f"Cal_{i}_Res = (NMR - Cal_{i}_Bmag)", inplace=True)
    df_fast.eval(f"Cal_{i}_Res_Rel = (NMR - Cal_{i}_Bmag) / NMR", inplace=True)

for i in range(1,6):
    for B in ["Bx","By","Bz"]:
        #df_fast.eval(f"Cal_{i}_Bmag = (Cal_{i}_Bx**2+Cal_{i}_By**2+Cal_{i}_Bz**2)**(1/2)", inplace=True)
        df_fast.eval(f"Cal_{i}_{B}_Res = (NMR - Cal_{i}_{B})", inplace=True)
        df_fast.eval(f"Cal_{i}_{B}_Res_Rel = (NMR - Cal_{i}_{B}) / NMR", inplace=True)

df_fast["seconds_delta"] = pd.to_timedelta(df_fast.index.values) / pd.offsets.Second(1) - 1565697533.416

df_slow["seconds_delta"] = pd.to_timedelta(df_slow.index.values) / pd.offsets.Second(1) - 1565697533.416''';

In [ ]:
#pd.to_timedelta(df_slow.index.values) / pd.offsets.Second(1)

In [ ]:
getsizeof(df_fast) / 1000 / 1000

In [ ]:
# relative residual

In [ ]:
for i in range(1,6):
    for B in ["Bx", "By", "Bz"]:
        fig = histo(df_fast[f"Cal_{i}_{B}_Res_Rel"], bins=50, show_plot=False, inline=True)
        filename = f"histo_Cal_{i}_{B}_Res_Rel"
        pio.write_image(fig, plotdir+f"relative_residuals/components/{filename}.pdf")
        pio.write_image(fig, plotdir+f"relative_residuals/components/{filename}.png")

In [ ]:
for i in range(1,6):
    fig = histo(df_fast[f"Cal_{i}_Res_Rel"], bins=50, show_plot=False, inline=True)
    filename = f"histo_Cal_{i}_Res_Rel"
    pio.write_image(fig, plotdir+f"relative_residuals/magnitude/{filename}.pdf")
    pio.write_image(fig, plotdir+f"relative_residuals/magnitude/{filename}.png")

In [ ]:
# residual

In [ ]:
for i in range(1,6):
    for B in ["Bx", "By", "Bz"]:
        fig = histo(df_fast[f"Cal_{i}_{B}_Res"], bins=50, show_plot=False, inline=True)
        filename = f"histo_Cal_{i}_{B}_Res"
        pio.write_image(fig, plotdir+f"residuals/components/{filename}.pdf")
        pio.write_image(fig, plotdir+f"residuals/components/{filename}.png")

In [ ]:
for i in range(1,6):
    fig = histo(df_fast[f"Cal_{i}_Res"], bins=50, show_plot=False, inline=True)
    filename = f"histo_Cal_{i}_Res"
    pio.write_image(fig, plotdir+f"residuals/magnitude/{filename}.pdf")
    pio.write_image(fig, plotdir+f"residuals/magnitude/{filename}.png")

In [ ]:
Ns = [5,10,30,60, 120, 240, 480, 960, 1930,1930*2, 1930*4, 1930*8]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_fast.loc[:, "Cal_1_Bmag_Avg"] = df_fast["Cal_1_Bmag"].rolling(window = N, center=True).mean()
    df_fast.loc[:, "Cal_1_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast["Cal_1_Bmag_Avg"])/df_fast["NMR"]
    df_fast_N.append(copy.copy(df_fast[N // 2::N]))


### Check Field Angle and Average
- Leaving for historical purposes, but column names have changed

In [ ]:
'''df_fast["Cal_1_Transverse"] = (df_fast["Cal_1_Bx"]**2+df_fast["Cal_1_By"]**2)**(1/2)
df_fast["theta"] = np.arccos(df_fast["Cal_1_Bz"]/df_fast["Cal_1_Bmag"])
df_fast["phi"] = np.arccos(df_fast["Cal_1_Bx"]/df_fast["Cal_1_Transverse"])''';

In [ ]:
fig = histo(df_fast.theta*180/np.pi, bins=50, inline=True)
fig = histo(df_fast.phi*180/np.pi, bins=50, inline=True)

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
Ns = [5,10,30,60, 120, 240, ]#480, 960, 1930,1930*2, 1930*4, 1930*8]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_fast.loc[:, "theta_Avg"] = df_fast["theta"].rolling(window = N, center=True).mean()
    df_fast.loc[:, "phi_Avg"] = df_fast["phi"].rolling(window = N, center=True).mean()
    df_fast_N.append(copy.copy(df_fast[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df["Cal_1_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])

In [ ]:
fig = histo([df_fast.theta]+[df.theta_Avg for df in df_fast_N], bins=50,same_bins=True, show_plot=False, opacity=0.5, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)

In [ ]:
fig = scatter2d(df_fast.index, df_fast.theta, inline=True)

In [ ]:
fig = scatter2d(df_fast.index, df_fast.phi, inline=True)

In [ ]:
df_fast_N[0].theta_Avg.plot()

In [ ]:
fig = scatter2d([df_fast.index, df_fast_N[2].index, df_slow.index], [df_fast.Cal_1_Bmag,df_fast_N[2].Cal_1_Bmag_Avg, df_slow[" NMR [T]"]], inline=True)

### Plots for Email

In [ ]:
# get probe name dictionary
directory = "/home/ckampa/Dropbox/LogFiles/"
filename_fast = '2019-08-13 1155830 hallonly.txt'

with open(directory+filename_fast, 'r') as file:
    lines = file.readlines()[1:6]

In [ ]:
lines

In [ ]:
id_reg = re.compile(": (.+?)  Offset")

In [ ]:
hall_ids_list = [id_reg.findall(line)[0] for line in lines]

In [ ]:
hall_ids_list

In [ ]:
hall_ids = {i+1: hall for i,hall in enumerate(hall_ids_list)}

In [ ]:
hall_ids

In [ ]:
fig = scatter2d(df_fast.index, df_fast["Raw_4_Bx"], inline=True)

In [ ]:
fig = scatter2d(df_fast.index, df_fast["Raw_4_Bz"], inline=True)

In [ ]:
fig = scatter2d(df_fast.index, df_fast["Cal_4_Bz"], inline=True)

In [ ]:
fig = scatter2d(5*[df_fast.index], [df_fast[f'Cal_{i}_T'] for i in range(1,6)])

In [ ]:
fig = scatter2d([df_fast.seconds_delta, df_slow.seconds_delta], [df_fast[f'Cal_1_T'], df_slow[" Hall Element"]],show_plot=False)

In [ ]:
figprof = py_profile(df_fast.seconds_delta, df_fast.Cal_1_T, x_bins=50, show_plot=False)

In [ ]:
figprof2 = py_profile(df_slow.seconds_delta, df_slow[" Hall Element"], x_bins=50,xrange=[df_fast.seconds_delta.min(),df_fast.seconds_delta.max()], show_plot=False)

In [ ]:
fig.add_traces([figprof.data[0],figprof2.data[0]])

In [ ]:
df_fast.loc[:,"HallElement"] = [df_slow.iloc[df_slow.index.get_loc(row, method='nearest')][" Hall Element"] for row in df_fast.index]

In [ ]:
#fig = scatter2d(df_fast.HallElement, df_fast.Cal_1_T, markers=True, lines=False)

In [ ]:
fig = py_profile(df_fast.HallElement, df_fast.Cal_1_T, x_bins=12)

In [ ]:
df_fast["Cal_4_Bmag"].describe()

In [ ]:
fig = scatter2d(df_fast.index, df_fast["Cal_1_Bmag"], inline=True)

In [ ]:
fig = scatter2d(df_slow_old.index, df_slow_old[" NMR [T]"], inline=True)

In [ ]:
fig = scatter2d(df_slow.index, df_slow[" NMR [T]"], inline=True)

In [ ]:
start = 0
end = 600
x_bins = 20

In [ ]:
fig = scatter2d(df_fast.seconds_delta[start:end], df_fast.Cal_1_Bmag[start:end],show_plot=False)

In [ ]:
figprof = py_profile(df_fast.seconds_delta[start:end], df_fast.Cal_1_Bmag[start:end], x_bins = x_bins, show_plot=False)

In [ ]:
fig.add_trace(figprof.data[0])

In [ ]:
#fig.update_traces(visible=False)

In [ ]:
pio.write_image(fig, plotdir+"scatter_plus_profile_test.pdf")

In [ ]:
start = 1000
end = 1600
x_bins = 20

comp_old = "Mag"
comp_new = "Bmag"

In [ ]:
# old data
fig = scatter2d(df_slow_old.seconds_delta[start:end], df_slow_old[f"Cal_{comp_old}"][start:end],show_plot=False)
figprof = py_profile(df_slow_old.seconds_delta[start:end], df_slow_old[f"Cal_{comp_old}"][start:end], x_bins = x_bins, show_plot=False)
fig.add_trace(figprof.data[0])

In [ ]:
for i in range(1,6):
    fig = scatter2d(df_fast.seconds_delta[start:end], df_fast[f'Cal_{i}_{comp_new}'][start:end],show_plot=False)
    figprof = py_profile(df_fast.seconds_delta[start:end], df_fast[f'Cal_{i}_{comp_new}'][start:end], x_bins = x_bins, show_plot=False)
    fig.add_trace(figprof.data[0])
    iplot(fig)

In [ ]:
start = 1000
end = 1120
x_bins = 6

In [ ]:
# old data
fig = scatter2d(df_slow_old.seconds_delta[start:end], df_slow_old[f"Cal_{comp_old}"][start:end],show_plot=False)
figprof = py_profile(df_slow_old.seconds_delta[start:end], df_slow_old[f"Cal_{comp_old}"][start:end], x_bins = x_bins, show_plot=False)
fig.add_trace(figprof.data[0])

In [ ]:
for i in range(1,6):
    fig = scatter2d(df_fast.seconds_delta[start:end], df_fast[f'Cal_{i}_{comp_new}'][start:end],show_plot=False)
    figprof = py_profile(df_fast.seconds_delta[start:end], df_fast[f'Cal_{i}_{comp_new}'][start:end], x_bins = x_bins, show_plot=False)
    fig.add_trace(figprof.data[0])
    iplot(fig)

In [ ]:
fig = histo(df_fast.Cal_1_Res_Rel, bins=50, show_plot=False)
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
iplot(fig)

In [ ]:
df_slow_old["Cal_Res_Rel"] = (df_slow_old[" NMR [T]"] - df_slow_old["Cal"]) / df_slow_old[" NMR [T]"]

In [ ]:
df_slow_old = df_slow_old.loc["2019-08-10 12:30:00":]

In [ ]:
fig = histo(df_slow_old.Cal_Mag_Res_Rel, bins=50, show_plot=False)
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
iplot(fig)

In [ ]:
df_slow_old.columns

In [ ]:
## AVERAGING

In [ ]:
import copy

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
Ns = [10,60]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_fast.loc[:, "Cal_1_Bmag_Avg"] = df_fast["Cal_1_Bmag"].rolling(window = N, center=True).mean()
    df_fast.loc[:, "Cal_1_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast["Cal_1_Bmag_Avg"])/df_fast["NMR"]
    df_fast_N.append(copy.copy(df_fast[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df["Cal_1_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])

In [ ]:
fig = histo([df_fast.Cal_1_Res_Rel]+[df.Cal_1_Bmag_Avg_Res_Rel for df in df_fast_N], bins=50,same_bins=False, show_plot=False, opacity=0.5, inline=True)
fig.update_traces(histnorm='probability density')
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
fig.layout.yaxis.title = ("Normalized Counts")
iplot(fig)

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
Ns = [10,60]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_slow_old.loc[:, "Cal_1_Bmag_Avg"] = df_slow_old["Cal_Mag"].rolling(window = N, center=True).mean()
    df_slow_old.loc[:, "Cal_1_Bmag_Avg_Res_Rel"] = (df_slow_old[" NMR [T]"] - df_slow_old["Cal_1_Bmag_Avg"])/df_slow_old[" NMR [T]"]
    df_fast_N.append(copy.copy(df_slow_old[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df["Cal_1_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])

In [ ]:
fig = histo([df_slow_old.Cal_Mag_Res_Rel]+[df.Cal_1_Bmag_Avg_Res_Rel for df in df_fast_N], bins=15,same_bins=False, show_plot=False, opacity=0.5, inline=True)
fig.update_traces(histnorm='probability density')
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
fig.layout.yaxis.title = ("Normalized Counts")
iplot(fig)

### Defining Profile Plot
- Generate TProfile, grab points and errors, replot in Plotly

In [ ]:
'''%%writefile /home/ckampa/Coding/py3/profile_plot.py

def profile(x,y, x_bins, xrange=None, show_plot=True, compare=False):#, show_root_plot=False):
    if xrange==None:
        xrange= [x.min(), x.max()]
    
    # Generate TProfile
    #del(tprof)
    gDirectory.GetList().Delete()
    tprof = TProfile('tprof', 'Profile Plot', x_bins, xrange[0], xrange[1])#, #1.249525, 1.24955)
    for xi,yi in zip(x,y):
        tprof.Fill(xi,yi)
    
    # Collect data into Pandas Series
    bin_centers = []
    bin_contents = []
    bin_errors = []

    for i in range(1,x_bins+1):
        bin_centers.append(tprof.GetBinCenter(i))
        bin_contents.append(tprof.GetBinContent(i))
        bin_errors.append(tprof.GetBinError(i))
    
    bin_centers = pd.Series(bin_centers, name=x.name)
    bin_contents = pd.Series(bin_contents, name=y.name)
    bin_errors = pd.Series(bin_errors, name="bin errors")
    
    x_error = tprof.GetBinWidth(1)/2
    
    # Plot with Plotly!
    name = (
        f'<br>{y.name}<br>'
        f'entries: {len(x)}<br>'
        f'mean: {x.mean():.3E}<br>'
        f'mean y: {y.mean():.3E}<br>'
        f'std dev: {x.std():.3E}<br>'
        f'std dev y: {y.std():.3E}'
        )
    
    fig = scatter2d(bin_centers, bin_contents, lines=False, markers=True, size_list=[1], 
                    title="Profile Plot: ",show_plot=False, inline=True)
    fig.update_traces(
        error_x=dict(type="constant", value=x_error, width=0),
        error_y=dict(type="data", array=bin_errors, width=0, visible=True),
        name=name,
    )
    
    if show_plot:
        iplot(fig)
    
    #if show_root_plot:
    #    c1 = TCanvas("c1", "Canvas", 200, 10, 700, 500)
    #    tprof.Draw()
    #    c1.Draw()
    if compare:
        return fig, tprof
    else:
        return fig''';

In [ ]:
## TESTING py_profile (in hpc_plots)

In [ ]:
figprof = py_profile(df_fast.seconds_delta, df_fast.Cal_1_Bmag, x_bins=2000, return_tprof=False, inline=True)

It works!

In [ ]:
# this was the comparison plot... must define function above if wanting to use again
#figprof = profile(df_fast.seconds_delta, df_fast.Cal_1_Bmag, x_bins=2000,) #show_root_plot=True)

In [ ]:
fig = scatter2d(df_fast.Cal_1_T, df_fast.Cal_1_Bmag, markers=True, lines=False, inline=True)

In [ ]:
profile(df_fast.Cal_1_T, df_fast.Cal_1_Bmag-df_fast.NMR, x_bins=100,);

In [ ]:
fig, tprof = profile(df_fast.Cal_1_T, df_fast.NMR, x_bins=100, compare=True)

In [ ]:
figprof, tprof = profile(df_fast.seconds_delta, df_fast.Cal_1_T, x_bins=2000,)

In [ ]:
fig = scatter2d(df_slow.seconds_delta, df_slow[" NMR [T]"], inline=True)

In [ ]:
#fig.add_trace(figprof.data[0])

In [ ]:
fig_cal_raw = scatter2d(df_fast.seconds_delta, df_fast.Cal_1_Bmag, inline=True)

In [ ]:
fig.add_traces([fig_cal_raw.data[0],figprof.data[0],])